In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import keras.layers as layers
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
import pickle
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten
import orjson

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
with open('dataset_test_files.pkl', 'rb') as fp:
    df_test = pickle.load(fp)
df_test.head()

,filename,kind,defect,size,class_name,class
3,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3
4,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3
11,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3
15,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3
20,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3


In [3]:
with open('dataset_train_files.pkl', 'rb') as fp:
    df_train = pickle.load(fp)
df_train.head()

,filename,kind,defect,size,class_name,class
0,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3
1,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3
2,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,M,RS,3
5,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,M,RS,3
6,D:\desenvolvimento\datasets\TRAIN1200 -1200_CO...,R,S,G,RS,3


In [4]:
gen = ImageDataGenerator()
gen

In [5]:
flow_test  = gen.flow_from_dataframe(df_test, y_col='class_name')
flow_train  = gen.flow_from_dataframe(df_train, y_col='class_name')
flow_test, flow_train

Found 360 validated image filenames belonging to 4 classes.
Found 840 validated image filenames belonging to 4 classes.


(<keras.preprocessing.image.DataFrameIterator at 0x1caf33d78c8>,
 <keras.preprocessing.image.DataFrameIterator at 0x1caf333c388>)

In [6]:
df_test[df_test['class'] != flow_test.classes]

,filename,kind,defect,size,class_name,class


In [7]:
df_train[df_train['class'] != flow_train.classes]

,filename,kind,defect,size,class_name,class


In [8]:
model = keras.Sequential()

model.add(Conv2D(16, (5, 5), activation='relu', input_shape=(256, 256, 3))) 
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu')) 
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='softmax'))

In [9]:
model.compile(optimizer='adam', # Rotina de otimização, que informa a ml como ajustar o valor dos parâmetros para minimizar o erro.
              loss='categorical_crossentropy', # Função de erro que diz o quão erradas estão nossas predições
              metrics=['accuracy']) # Lista de métricas para avaliar o nosso modelo

In [10]:
with open('model_raw.json', 'w') as fp:
    fp.write(model.to_json())

In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 252, 252, 16)      1216      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 126, 126, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 124, 124, 32)      4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 60, 60, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 16)       

In [12]:
history = model.fit_generator(flow_train, epochs=100, verbose=2, validation_data=flow_test, callbacks=[
    ModelCheckpoint(
        'model_raw.{epoch:03d}-{val_accuracy:.4f}.h5',
        monitor='val_accuracy',
        save_best_only=True,
        save_weights_only=True,
    ),
    TensorBoard(log_dir='logs_raw')
])

Epoch 1/100
 - 3s - loss: 4.0885 - accuracy: 0.2607 - val_loss: 1.3609 - val_accuracy: 0.3361
Epoch 2/100
 - 2s - loss: 1.3044 - accuracy: 0.4060 - val_loss: 1.2314 - val_accuracy: 0.4083
Epoch 3/100
 - 3s - loss: 1.1619 - accuracy: 0.4893 - val_loss: 1.3461 - val_accuracy: 0.5111
Epoch 4/100
 - 3s - loss: 1.0727 - accuracy: 0.5202 - val_loss: 1.1196 - val_accuracy: 0.5250
Epoch 5/100
 - 3s - loss: 1.0305 - accuracy: 0.5714 - val_loss: 0.8716 - val_accuracy: 0.5389
Epoch 6/100
 - 3s - loss: 0.9437 - accuracy: 0.5964 - val_loss: 1.4271 - val_accuracy: 0.5361
Epoch 7/100
 - 3s - loss: 0.8861 - accuracy: 0.6405 - val_loss: 0.7699 - val_accuracy: 0.5583
Epoch 8/100
 - 3s - loss: 0.8953 - accuracy: 0.6512 - val_loss: 0.6956 - val_accuracy: 0.5389
Epoch 9/100
 - 2s - loss: 0.7718 - accuracy: 0.6881 - val_loss: 0.8070 - val_accuracy: 0.5333
Epoch 10/100
 - 3s - loss: 0.6855 - accuracy: 0.7155 - val_loss: 0.6961 - val_accuracy: 0.5806
Epoch 11/100
 - 3s - loss: 0.6613 - accuracy: 0.7464 - val_

In [13]:
with open('train_raw_history.json', 'wb') as fp:
    fp.write(orjson.dumps({k: np.array(v).tolist() for k, v in history.history.items()}))